# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [6]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一个平静且自然的景观。图片中心是一条木制的步道，直通前方，周围是郁郁葱葱的高草和一些灌木。这条步道可能是为了让人们可以轻松地穿过湿地或草地而建。图像的背景是广阔的天空，蓝天中散布着一些白云，给人一种宁静和自由的感觉。\n\n这个场景带有强烈的自然和平静的氛围，适合那些寻求从日常忙碌生活中逃离，享受自然之美的人。步道引人向前，激发了探索和冒险的欲望，而周围的自然环境提供了一个放松和沉思的完美空间。', role='assistant', function_call=None, tool_calls=None))


In [7]:
response.choices[0].message.content

'这幅图展示了一个平静且自然的景观。图片中心是一条木制的步道，直通前方，周围是郁郁葱葱的高草和一些灌木。这条步道可能是为了让人们可以轻松地穿过湿地或草地而建。图像的背景是广阔的天空，蓝天中散布着一些白云，给人一种宁静和自由的感觉。\n\n这个场景带有强烈的自然和平静的氛围，适合那些寻求从日常忙碌生活中逃离，享受自然之美的人。步道引人向前，激发了探索和冒险的欲望，而周围的自然环境提供了一个放松和沉思的完美空间。'

### 封装成一个函数 query_image_description

In [8]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [9]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默的对比图片，显示了两种截然不同的狗狗形象。左边是一只狗被赋予了人类肌肉健壮的身体，标注“16岁的我”，下方有文字“我刚走一片光明，岁月未央，人山人海的天底下”。右边是一只看起来悠闲自得、体形稍显圆润的狗狗，标注“工作后的我”，下面的文字则是“好累啊都要瘫倒坐骨神经痛，巴不得不用骨折裂花钱看病，我在顾小四购”。

这类图片常用于表现人们在年轻时候的理想与成年后的现实之间的差异，通过夸张和幽默的手法反映出生活的压力和变化。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [1]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [11]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

该图为1980年至2020年间美国、中国、日本和德国的国内生产总值（GDP）比较图。

1. **美国（蓝色线）**：展示了美国GDP从1980年以后的持续和稳定增长。从图中可以看出，美国经济总体保持上升趋势，至2020年接近20万亿美元。

2. **中国（红色线）**：从1980年的低起点开始，中国的GDP增长非常显著，特别是2000年之后增长速度加快，到2020年几乎达到了15万亿美元，几乎接近美国。

3. **日本（紫色线）**：日本的GDP在1980年代和1990年代初期增长迅速，但在1990年代中后期达到高峰后增长放缓，之后基本保持平稳，在5万亿美元到6万亿美元之间波动。

4. **德国（绿色线）**：德国的GDP增长比较平稳，总体呈缓慢上升趋势，到2020年接近4万亿美元。

总的来说，这幅图表展示了这四个经济体在过去40年的经济表现，其中中国的快速增长尤为引人注目，而美国则保持了较高的经济总量和稳定增长。日本和德国的经济增长相对较为平缓。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [14]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片显示的是一本笔记本上关于自然语言处理技术的笔记。内容主要涉及几种不同的调优方法用于预训练语言模型，如Prompt Tuning、Prefix Tuning，以及LoRA调整技术。

具体来说：
1. **Prompt Tuning（提示调整）**：这种方法通常用于相对较小的模型，通过在输入端增加一些固定的标记（tokens），帮助模型更好地理解和处理任务。笔记中提到的 `X = [X1, X2, ..., Xn]` 表示输入的嵌入向量，`W` 和 `X'` 分别表示权重和转换后的输入向量。

2. **Prefix Tuning（前缀调整）**：这种技术通过在模型输入的序列前面加上一系列固定的向量（即前缀）来影响模型的行为，使得模型输出 `Y` 由 `W` 和通过前缀向量修改后的 `X` 计算得出。

3. **LoRA（Low-Rank Adaptation，低秩适应）**：这是一种通过修改模型的权重来进行调整的方法。它通过添加微小的矩阵调整来改变原有模型的权重矩阵 `W`。这里的 `ΔW = A·B` 表示权重的增量，其中 `A` 和 `B` 是低秩矩阵，用于减少参数数量并节省计算资源。

笔记还提到了几个实际的存储需求比较，如LoRA和其它技术在处理模型时所需的存储空间。

这些内容显示了深度学习和自然语言处理领域中的一些先进技术和策略。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [15]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一本笔记本上关于自然语言处理技术的笔记。内容主要涉及几种不同的调优方法用于预训练语言模型，如Prompt Tuning、Prefix Tuning，以及LoRA调整技术。

具体来说：
1. **Prompt Tuning（提示调整）**：这种方法通常用于相对较小的模型，通过在输入端增加一些固定的标记（tokens），帮助模型更好地理解和处理任务。笔记中提到的 `X = [X1, X2, ..., Xn]` 表示输入的嵌入向量，`W` 和 `X'` 分别表示权重和转换后的输入向量。

2. **Prefix Tuning（前缀调整）**：这种技术通过在模型输入的序列前面加上一系列固定的向量（即前缀）来影响模型的行为，使得模型输出 `Y` 由 `W` 和通过前缀向量修改后的 `X` 计算得出。

3. **LoRA（Low-Rank Adaptation，低秩适应）**：这是一种通过修改模型的权重来进行调整的方法。它通过添加微小的矩阵调整来改变原有模型的权重矩阵 `W`。这里的 `ΔW = A·B` 表示权重的增量，其中 `A` 和 `B` 是低秩矩阵，用于减少参数数量并节省计算资源。

笔记还提到了几个实际的存储需求比较，如LoRA和其它技术在处理模型时所需的存储空间。

这些内容显示了深度学习和自然语言处理领域中的一些先进技术和策略。

![](./images/handwriting_1.jpg)

In [16]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片显示的是一份笔记，内容关注于计算机科学中的自然语言处理（NLP），特别是关于变换模型（Transformers）、调整技术和最新方法。这里面包括了一系列的技术和方法名，如：

1. **PEFT/SOTA 和 PBFT Methods** - 这些可能是针对特定模型或任务的前沿技术或方法。
2. **Prompt Tuning** - 提到了不同类型的 Prompt Tuning 方法, 如:
   - **Adapter (2019, Google)**
   - **Prefix (2021, Stanford)**
   - **Prompt (2021, Google)**
   - **P-Tuning V1/V2 (2021)**
   - **Soft Prompts (2021)**
   - **Hand Prompts - template-based**

3. **Instruction Tuning** - 指导型调整技术，可能用于改进语言模型。
4. **LoRA, PETC, I3** - 这些也是最新的调整技术或模型改进手段。

右侧笔记提到：
- **多模态指令微调 (Multi-modality Instruction FT)**，以及**LLaMA（LLaMA(3B)）**
- **Adaptors和Prefix-tuning的比较**
- **MAM Adaptors** - 一个可能是模型扩展方法，强调缩放并行和优化结果。

这些内容涵盖了多种先进技术，专门用于改善或自定义大型语言模型（LLMs）的性能，使其更适合特定的应用或任务。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。
![](./images/gpt-4v.jpg)

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [18]:
from IPython.display import display, Markdown
def display_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):
    content = query_base64_image_description(image_path, prompt, max_tokens)
    display(Markdown(content))

# Describe the image
display_image_description("./images/gpt-4v.jpg")

这张图片展示的是一段关于统计物理或热力学中的数学理论部分的说明。内容大致介绍了一个系统的能量变化模型，包含数学表达式和理论推导。具体分为几个部分：

1. 初始条件和假设。 
   - 初态时刻 \( t = T_0 \)，系统的能量为 \( E(x_0) \)。
   - 对于 \( t = kT \) 时刻， \( k \) 在区间 \( (0,1) \) 内，且 \( k \) 可以是非整数。
   - 描述了能量在状态 \( x_t \) 和 \( x_{t+1} \) 之间的变化量 \( \Delta E = E(x_{t+1}) - E(x_t) \)。

2. 能量变化的模型。
   - 提及了当 \( \Delta E < 0 \) 时，状态变化符合一定概率分布，特定形式为 \( e^{-\Delta E / kT} \)，这是统计物理学中常见的 Boltzmann 分布式描述。

3. 概率的计算和使用。
   - 在模型中，使用了特定的概率计算方法来估算在不同能量状态之间的转换概率。

从文中内容看，这可能是从一本高等物理学或统计力学的教材中摘录的一部分，专门讨论系统状态之间的转换和相应的能量变化，这是这些领域研究中的一个基本问题。

In [2]:
# Describe the image with a custom prompt
display_image_description("./images/gpt-4v.jpg", prompt="请说出图中描述的算法和推导过程")

图中描述的是模拟退火算法的基本步骤和计算过程。模拟退火算法是一种概率性算法，用于求解全局优化问题，其灵感来源于材料科学中金属退火的过程。以下是对算法的基本描述：

1. 初始化温度为 \( T_0 \)（一开始的高温），并设置当前解为随机生成的初始解 \( x_0 \)，计算该解的目标函数值 \( E(x_0) \)。

2. 在每一次迭代中，降低温度 \( T \) 直到 \( T \) 达到终止温度 \( T_f \)，过程中重复以下步骤：
   - 随机生成一个新的解 \( x_{t+1} \)，通常通过在当前解 \( x_t \) 基础上进行小的随机扰动来实现。
   - 计算新解 \( x_{t+1} \) 的目标函数值 \( E(x_{t+1}) \)。
   - 计算目标函数值的变化 \(\Delta E = E(x_{t+1}) - E(x_t)\)。
   - 如果 \(\Delta E < 0\)，即新解比当前解更优，接受新解为当前解。
   - 如果 \(\Delta E \geq 0\)，则以一定的概率 \( e^{-\Delta E / kT} \) 接受较差的解作为当前解，这里 \( k \) 是控制参数。

3. 逐步降低温度 \( T \) 并重复上述步骤直到温度低于预设的终止温度 \( T_f \)，通常 \( T \) 的更新通过乘以一个冷却因子来进行（例如乘以 0.9）。

4. 输出当前解 \( x_t \) 作为问题的解。

图中还提到了关于如何选择初始温度、冷却率、终止温度以及迭代次数，这些都是实施模拟退火算法时的重要参数选择。通过适当调整这些参数，可以影响算法的收敛性和求解质量。